In [2]:
# korQuad 파일을 불러옴.
jsonfile = '../../../data11/ai_hub/qua/VL_span_inference.json'
contexts, questions, answers, contextids, qcontextids = read_aihub_VL_text_json(jsonfile)

In [5]:

import numpy as np
import pandas as pd
# list들을 zip 으로 묶고, dataframe 생성함
# context, contextid를 묶어서 context df 만듬.
df_contexts = pd.DataFrame((zip(contexts, contextids)), columns = ['context','contextid'])

# question, answer, contextids를 묶어서 question df 만듬
df_questions = pd.DataFrame((zip(questions, answers, qcontextids)), columns = ['question','answer', 'contextid'])

In [6]:
df_contexts.head()

,context,contextid
0,대구도시철도공사가 한국생산성본부 국가고객만족도(NCSI) 조사에서 13년 연속으로 ...,10001
1,국민의힘이 4·7 재·보궐선거에서 승리한 이유가 국민의힘의 정책이나 공약이 좋아서가...,10002
2,홍남기 부총리 겸 기획재정부 장관이 오세훈 서울시장 취임 후 상승폭을 키운 부동산 ...,10003
3,배우 이일화가 ‘나의 너에게’에 특별출연한다.\n빅보스엔터테인먼트는 15일 “이일화...,10004
4,“가짜 뉴스 홍보로 도민 혈세를 낭비하고 갈등을 조장하는 원희룡 제주지사는 제주를 ...,10005


In [7]:
df_questions.head()

,question,answer,contextid
0,13년 동안 국가고객만족도에 1위를 한 공기업의 사장은 누구야,홍승활,10001
1,국가고객만족도에서 13년간 일등을 한 공기업의 사장은 누구야,홍승활,10001
2,국힘이 재보궐선거에서 이긴 이유를 분석한 자세한 내용은 어디서 볼 수 있어,전국지표조사(NBS) 또는 중앙선거여론조사심의위원회 홈페이지,10002
3,국힘이 재보궐선거에서 이긴 이유를 분석한 자세한 내용은 어떤 곳에서 볼 수 있어,전국지표조사(NBS) 또는 중앙선거여론조사심의위원회 홈페이지,10002
4,SNS에 서울의 부동산 현황에 대해 걱정된다고 글을 누가 작성했어,홍남기,10003


In [ ]:
#============================================================================
# BM25 예제임
# => 
#============================================================================
# bm25 설치
# !pip install rank_bm25
import konlpy
from konlpy.tag import Mecab
from rank_bm25 import BM25Okapi
from tqdm.notebook import tqdm


contexts = [
    "세계 배달 피자 리더 도미노피자가 우리 고구마를 활용한 신메뉴를 출시한다.도미노피자는 오는 2월 1일 국내산 고구마와 4가지 치즈가 어우러진 신메뉴 `우리 고구마 피자`를 출시하고 전 매장에서 판매를 시작한다. 이번에 도미노피자가 내놓은 신메뉴 `우리 고구마 피자`는 까다롭게 엄선한 국내산 고구마를 무스와 큐브 형태로 듬뿍 올리고, 모차렐라, 카망베르, 체더 치즈와 리코타 치즈 소스 등 4가지 치즈와 와규 크럼블을 더한 프리미엄 고구마 피자다.",
    "피자의 발상지이자 원조라고 할 수 있는 남부의 나폴리식 피자(Pizza Napolitana)는 재료 본연의 맛에 집중하여 뛰어난 식감을 자랑한다. 대표적인 나폴리 피자로는 피자 마리나라(Pizza Marinara)와 피자 마르게리타(Pizza Margherita)가 있다.",
    "도미노피자가 삼일절을 맞아 '방문포장 1+1' 이벤트를 진행한다. 이번 이벤트는 도미노피자 102개 매장에서 3월 1일 단 하루 동안 방문포장 온라인, 오프라인 주문 시 피자 1판을 더 증정하는 이벤트다. 온라인 주문 시 장바구니에 2판을 담은 후 할인 적용이 가능하며, 동일 가격 또는 낮은 가격의 피자를 고객이 선택하면 무료로 증정한다."
]

def tokenizer(sent):
  return sent.split(" ")

# 입력된 contexts를 mecab을 이용하여 형태소 추출 후 " " 붙여서 형태소 문장을 만듬.
# Mecab 선언
mecab = Mecab()

mecab_contexts=[]
for context in tqdm(contexts):
    temp = mecab.morphs(context)   # ['세계', '배달', '피자', '리더', '도미노피자','가'..] 식으로 temp 리스트가 생성됨
    #print(temp)
    sentence = " ".join(temp)      # 위 temp 리스트를 공백을 넣어서 한문장으로 합침 ['세계 배달 피자 리더 도미노피자 가 ...]
    mecab_contexts.append(sentence)
    
print(mecab_contexts)
print(len(mecab_contexts))        


# tokeniaer 후 인덱싱
tokenized_corpus = [tokenizer(doc) for doc in mecab_contexts]
bm25 = BM25Okapi(tokenized_corpus)

print(bm25.doc_len)
print(bm25.doc_freqs)

In [ ]:
#idf : 토큰의 inverse term frequency를 계산해둠
# bm25.idf

In [ ]:
query = "고구마 피자 신메뉴"
tokenized_query = tokenizer(query)

# 쿼리에 따른 스코어 구함
doc_scores = bm25.get_scores(tokenized_query)
doc_scores

In [ ]:
# 쿼리 문장 구함
bm25.get_top_n(tokenized_query, mecab_contexts, n=2)

In [ ]:
contexts = [
'제주도 제주 서귀포 제주한라산 제주맛집휴계소 비행기',
'서울특별시 서울에버랜드 서울 관광지',
'부산해운대 부산 광역시 부산관광지 부산맛집',
'강원도 겨울 강원도 레고랜드 강원도 휴계소',
'파리 프랑스 파리 에펠탑관광지 비행기'
]

user_query = [
'서울 관광지',
'제주 맛집',
'에펠탑 관광'
]

ground_truths_list = [1,0,4]
contextids = [0]

search_k = 3

In [ ]:
    import konlpy
    from konlpy.tag import Mecab
    from rank_bm25 import BM25Okapi
    from tqdm.notebook import tqdm
    
    from os import sys
    sys.path.append('../../')
    from myutils import mean_reciprocal_rank
    
    def BM25tokenizer(sent):
      return sent.split(" ")

    # 입력된 contexts를 mecab을 이용하여 형태소 추출 후 " " 붙여서 형태소 문장을 만듬.
    # Mecab 선언
    mecab = Mecab()

    mecab_contexts=[]
    for context in tqdm(contexts):
        temp = mecab.morphs(context)   # ['세계', '배달', '피자', '리더', '도미노피자','가'..] 식으로 temp 리스트가 생성됨
        sentence = " ".join(temp)      # 위 temp 리스트를 공백을 넣어서 한문장으로 합침 ['세계 배달 피자 리더 도미노피자 가 ...]
        mecab_contexts.append(sentence)

    print(f'*contexts_len:{len(mecab_contexts)}')  
    print(f'{mecab_contexts}')

    # tokeniaer 후 인덱싱
    tokenized_corpus = [BM25tokenizer(doc) for doc in mecab_contexts]
    bm25 = BM25Okapi(tokenized_corpus)

    print(f'bm25.doc_len:{bm25.doc_len}')
    print(f'type(bm25.doc_freqs):{type(bm25.doc_freqs)}')

    # 쿼리 후 get_scores 를 이용하여, scores를 구함.
    bm5_predictions_list = []

    for idx, query in enumerate(user_query):
        
        # 쿼리에 따른 스코어 구함
        tempq = mecab.morphs(query) 
        query_mecab = " ".join(tempq)
        tokenized_query = BM25tokenizer(query_mecab)
        print(tokenized_query)
        doc_scores = bm25.get_scores(tokenized_query)
        print(doc_scores)
        # 정렬후 최대 스코어 search_k 만큼만 출력함
        top_lists = sorted(enumerate(doc_scores), key=lambda x: x[1], reverse=True)[:search_k]
        bm5_predictions_list.append([index + contextids[0] for index, score in top_lists])
        print(top_lists)

    # 정답, 여기서는 contextid를 리스트로 만듬.
    #ground_truths_list = df_questions['contextid'].values.tolist()

    # MPR 계산
    predictions_list = bm5_predictions_list  # 예측 결과 리스트
    bm25_ranks, bm25_score = mean_reciprocal_rank(ground_truths_list, predictions_list)

     # BM25-MRR 출력
    print(f'--------------------------------------------------------------------------')
    print('*BM25-MRR:{:.4f}'.format(bm25_score))
    print(f'*Ranks({len(bm25_ranks)}):{bm25_ranks[0:20]}')
    # 검색 못한 계슈
    print(f'---------------------------------------------------------------------------')
    zero_count = 0
    for item in bm25_ranks:
        if item == 0:
            zero_count += 1
    
    print('*BM25 검색실패계수: {}/{}({:.2f}%)'.format(zero_count, len(bm25_ranks), (zero_count/len(bm25_ranks))*100))
    print(f'---------------------------------------------------------------------------')